# Exploring OpenStreetMap POIs with the Overpass Python wrapper and Pandas - Part 2

## Background

I created this notebook to help me explore OpenStreetMap Point of Interest (POI) data in a small area (and to improve my beginner's knowledge of Python and Pandas). It uses the [Overpass API Python wrapper](https://github.com/mvexel/overpass-api-python-wrapper) to obtain data from OpenStreetMap (OSM).  

In Part 1 I worked out how to get data from Overpass in the format I needed for a single point in time. Here I will extend this to comparisons at different dates.

Please see the [README](README.md) for more information.

## Retrieving data at two points in time

Import the same packages as in Part 1:

In [1]:
import overpass
import pandas as pd
import plotly.express as px
import os

In the previous notebook, I only requested *current* data from the Overpass API, so the query results reflected the state of the OpenStreetMap database at the moment I ran the query. But Overpass also allows access to the entire previous history of OSM objects (including objects that have been deleted). This is called "Attic Data", and I'll need it to compare my POI data at different points in time.

I'm going to wrap up the code from the previous notebook inside a function, and add an optional parameter to indicated the date of the Attic Data. If the parameter is not passed, the query will return current data.

In [2]:
def get_POIs(attic_date=''):

    # set timeout
    # queries on current data should be fast, but attic queries can take longer
    if attic_date == '':
        api_timeout = (20, 20)  # set both connection timeout and read timeout
    else:
        api_timeout = (60, 60)

    # initialize the API
    api = overpass.API(timeout=api_timeout)

    # include center coordinates for ways and relations
    api_verbosity = 'center'

    # specify administrative areas to search and construct string
    areas = [
        '9764340',  # Cristo de la Epidemia
        '9764345',  # La Victoria
        '9764343',  # Barcenillas
        '9764342',  # Conde de Ureña
    ]
    areas_str = ',\n'.join(areas)
    areas_str = f'rel(id:\n{areas_str});\nmap_to_area->.searchArea;\n'

    # specify features to include and construct string
    feature_keys = [
        'shop',
        'healthcare',
        'amenity',
        'office',
        'craft',
        'tourism',
        'leisure'
    ]
    features_formatted = (
        f'nwr(area.searchArea)["{feature_key}"]' for feature_key in feature_keys
    )
    features_str = ';\n'.join(features_formatted)
    features_str = f'(\n{features_str};\n);'

    api_query = areas_str + features_str

    # list output fields, add feature keys to list, and construct format string
    keys = [
        'name',
        '"addr:street"',
        '"addr:place"',
        '"addr:housenumber"',
        '"addr:postcode"',
        'brand',
        '"brand:wikidata"',
        'check_date',
        '::id',
        '::type',
        '::lat',
        '::lon'
    ] + feature_keys
    api_format = ',\n'.join(keys)
    api_format = f'csv({api_format})'

    # get data from the API
    data = api.get(api_query, responseformat=api_format,
                   verbosity=api_verbosity, date=attic_date)

    # create DataFrame using the first row of the query response as column names
    headers = data[0]
    del (data[0])
    df = pd.DataFrame(data, columns=headers)

    # map feature key/values to a single key/value combination
    def get_feature(row):
        # set feature key and value based on the first non-empty feature key
        for key in feature_keys:
            if row[key] != '':
                return key, row[key]
        return '', ''

    # create and populate the new key/value columns, dropping the originals
    df.insert(0, 'feature_key', '')
    df.insert(1, 'feature_value', '')
    df[['feature_key', 'feature_value']] = df.apply(get_feature,
                                                    axis=1, result_type='expand')
    df.drop(columns=feature_keys, inplace=True)

    # create "feature group"
    # (higher level than feature key/value, but more granular than key alone)
    feature_group_mapping = {
        # main groups follow "Beautified JOSM" presets
        ('amenity', 'restaurant'): 'amenity - food+drinks',
        ('amenity', 'fast_food'): 'amenity - food+drinks',
        ('amenity', 'food_court'): 'amenity - food+drinks',
        ('amenity', 'cafe'): 'amenity - food+drinks',
        ('amenity', 'ice_cream'): 'amenity - food+drinks',
        ('amenity', 'pub'): 'amenity - food+drinks',
        ('amenity', 'biergarten'): 'amenity - food+drinks',
        ('amenity', 'bar'): 'amenity - food+drinks',
        ('amenity', 'bench'): 'minor',
        ('amenity', 'drinking_water'): 'minor',
        ('amenity', 'fountain'): 'minor',
        ('amenity', 'parking'): 'minor',
        ('amenity', 'parking_entrance'): 'minor',
        ('amenity', 'parking_space'): 'minor',
        ('amenity', 'bicycle_parking'): 'minor',
        ('amenity', 'motorcycle_parking'): 'minor',
        ('amenity', 'recycling'): 'minor',
        ('amenity', 'waste_basket'): 'minor',
        ('amenity', 'waste_disposal'): 'minor',
        ('shop', 'supermarket'): 'shop - food',
        ('shop', 'convenience'): 'shop - food',
        ('shop', 'general'): 'shop - food',
        ('shop', 'kiosk'): 'shop - food',
        ('shop', 'bakery'): 'shop - food',
        ('shop', 'butcher'): 'shop - food',
        ('shop', 'seafood'): 'shop - food',
        ('shop', 'dairy'): 'shop - food',
        ('shop', 'cheese'): 'shop - food',
        ('shop', 'frozen_food'): 'shop - food',
        ('shop', 'deli'): 'shop - food',
        ('shop', 'pastry'): 'shop - food',
        ('shop', 'confectionery'): 'shop - food',
        ('shop', 'chocolate'): 'shop - food',
        ('shop', 'tea'): 'shop - food',
        ('shop', 'coffee'): 'shop - food',
        ('shop', 'herbalist'): 'shop - food',
        ('shop', 'greengrocer'): 'shop - food',
        ('shop', 'organic'): 'shop - food',
        ('shop', 'water'): 'shop - food',
        ('shop', 'alcohol'): 'shop - food',
        ('shop', 'beverages'): 'shop - food',
        ('shop', 'wine'): 'shop - food',
        ('shop', 'nutrition_supplements'): 'shop - food',
        ('shop', 'health_food'): 'shop - food',
        ('shop', 'spices'): 'shop - food',
        ('shop', 'hairdresser'): 'shop - beauty',
        ('shop', 'beauty'): 'shop - beauty',
        ('shop', 'clothes'): 'shop - clothes',
        ('shop', 'boutique'): 'shop - clothes',
        ('shop', 'shoes'): 'shop - clothes',
        ('shop', 'shoe_repair'): 'shop - clothes',
        ('shop', 'outdoor'): 'shop - clothes',
        ('shop', 'dry_cleaning'): 'shop - clothes',
        ('shop', 'laundry'): 'shop - clothes',
        ('shop', 'tailor'): 'shop - clothes',
        ('shop', 'fabric'): 'shop - clothes',
        ('shop', 'chemist'): 'shop - for the body',
        ('shop', 'cosmetics'): 'shop - for the body',
        ('shop', 'perfumery'): 'shop - for the body',
        ('shop', 'beauty'): 'shop - for the body',
        ('shop', 'massage'): 'shop - for the body',
        ('shop', 'tobacco'): 'shop - for the body',
        ('shop', 'e-cigarette'): 'shop - for the body',
        ('shop', 'hairdresser'): 'shop - for the body',
        ('shop', 'tattoo'): 'shop - for the body',
        ('shop', 'optician'): 'shop - for the body',
        ('shop', 'hearing_aids'): 'shop - for the body',
        ('shop', 'medical_supply'): 'shop - for the body',
        ('shop', 'jewelry'): 'shop - for the body',
        ('shop', 'erotic'): 'shop - for the body',
        ('leisure', 'garden'): 'minor',
        ('leisure', 'pitch'): 'minor',
        ('leisure', 'swimming_pool'): 'minor',
        ('tourism', 'artwork'): 'minor'
    }

    # function to apply the mapping
    def get_feature_group(row):
        # map key and value to feature_group
        # default to ' - general' if not specifically mapped
        key = row['feature_key']
        value = row['feature_value']
        return feature_group_mapping.get((key, value), f'{key} - general')

    # Apply the mapping to populate feature_group for each row
    df.insert(1, 'feature_group',  '')
    df['feature_group'] = df.apply(get_feature_group, axis=1)

    # drop minor features
    df = df[~(df['feature_group'] == 'minor')]

    # remove @ from names
    df.rename(columns={'@id': 'id', '@type': 'type',
                       '@lat': 'lat', '@lon': 'lon'}, inplace=True)

    # set type of non string columns
    df['lat'] = pd.to_numeric(df['lat'])
    df['lon'] = pd.to_numeric(df['lon'])
    df['check_date'] = pd.to_datetime(df['check_date'])

    # add column for street or place
    # populate with addr:street except where addr:place exists
    df.insert(8, 'addr:street_or_place', '')
    df['addr:street_or_place'] = df['addr:street'].mask(
        ~(df['addr:place'] == ''), df['addr:place'])
    # drop the old columns
    df.drop(columns=['addr:place', 'addr:street'], inplace=True)

    # create index and OSM link column
    df['full_id'] = df['type'] + '/' + df['id']
    df['osm_link'] = 'https://www.openstreetmap.org/' + df['full_id']
    df.set_index('full_id', inplace=True)

    # drop the columns now captured by the index
    df.drop(columns=['id', 'type'], inplace=True)

    return df

I'm going to query for current data first. I could call get_POIs without a date, but the results might change if anyone edits the map while I am working on this. So I'm going to set a date to ensure I get the same data as in Part 1.

In [3]:
date_new = '2024-10-29'
df_new = get_POIs(date_new)
len(df_new)

392

As expected, I have 392 POIs. The time taken by the API query has increased quite a lot compared to the query on current data, so I would only include a date when looking at recent data if I specifically need a snapshot at a fixed point in time.

The analysis of "check_date" in Part 1 showed that my intensive updating of POIs in this area started in late August. To see how that activity has affected the data, I'm going to compare POIs at dates two months apart. Of course the comparison may also reflect real changes such as shops opening and closing. But over such a short period, during which I was actively mapping POIs, those real changes are likely to account for a relatively small share of changes in the data.

In [4]:
date_old = '2024-08-15'
df_old = get_POIs(date_old)
len(df_old)

267

As I expected given that I added quite a few new POIs recently, there are fewer rows in the older data (254 versus 376).

Now to set up some folder and file names:

In [5]:
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)
output_file = 'compare-data-' + date_old + '-to-' + date_new
output_file_csv = os.path.join(output_folder, output_file + '.csv')
output_file_html = os.path.join(output_folder, output_file + '.html')

## Comparing tag frequency

Define a function to create a DataFrame summarising tag counts as in Part 1, and call it with both old and new data:  

In [6]:
def get_tag_summary(df):
    df_tags = pd.DataFrame(
        df[['feature_key', 'feature_group', 'feature_value']]
        .value_counts().reset_index())

    df_tags.rename(columns={'count': 'frequency'}, inplace=True)

    total = df_tags['frequency'].sum()
    df_tags['frequency_pc'] = (df_tags['frequency'] / total) * 100

    return df_tags


df_tags_new = get_tag_summary(df_new)
df_tags_old = get_tag_summary(df_old)

print(len(df_tags_new))
print()
print(len(df_tags_old))
print()
print(df_tags_new.head())
print()
print(df_tags_old.head())

121

96

  feature_key          feature_group feature_value  frequency  frequency_pc
0        shop    shop - for the body   hairdresser         29      7.397959
1     amenity  amenity - food+drinks          cafe         19      4.846939
2     amenity  amenity - food+drinks    restaurant         17      4.336735
3     amenity  amenity - food+drinks     fast_food         14      3.571429
4        shop            shop - food   greengrocer         12      3.061224

  feature_key          feature_group feature_value  frequency  frequency_pc
0     amenity  amenity - food+drinks    restaurant         20      7.490637
1     amenity  amenity - food+drinks          cafe         15      5.617978
2        shop         shop - clothes       clothes         13      4.868914
3        shop            shop - food   convenience         12      4.494382
4        shop    shop - for the body   hairdresser          9      3.370787


It seems that my intensive mapping campaign has increased the number of unique tags in the data by 25. And there's a strong hint that it has also changed the relative proportions of mapped POIs: Previously "amenity = restaurant" was the most common POI, but it has now been overtaken by "shop=hairdresser", and by some distance. That agrees with my subjective impression that I mapped a lot of previous missing hairdressers. 

Now I can join the two tag summary DataFrames, usign an outer join so that I have a row for every tag that appears in either dataset:

In [7]:
df_tags = pd.merge(df_tags_new, df_tags_old, how='outer', on=[
                   'feature_key', 'feature_group', 'feature_value'],
                   sort=True, suffixes=('_new', '_old'))
df_tags.fillna(0, inplace=True)
df_tags['freq_diff'] = df_tags['frequency_new'] - df_tags['frequency_old']

df_tags.head()

,feature_key,feature_group,feature_value,frequency_new,frequency_pc_new,frequency_old,frequency_pc_old,freq_diff
0,amenity,amenity - food+drinks,bar,2.0,0.510204,2.0,0.749064,0.0
1,amenity,amenity - food+drinks,cafe,19.0,4.846939,15.0,5.617978,4.0
2,amenity,amenity - food+drinks,fast_food,14.0,3.571429,8.0,2.996255,6.0
3,amenity,amenity - food+drinks,ice_cream,2.0,0.510204,3.0,1.123596,-1.0
4,amenity,amenity - food+drinks,pub,2.0,0.510204,1.0,0.374532,1.0


In [8]:
print(len(df_tags[df_tags['frequency_new'] == 0]))
print()
print(len(df_tags[df_tags['frequency_old'] == 0]))

9

34


I can now see that the net increase of 25 tags consists of 34 new tags less 9 that have disappeared. I'd better have a look at the disappeared tags to make sure nothing odd is happening:

In [9]:
df_tags[df_tags['frequency_new'] == 0]

,feature_key,feature_group,feature_value,frequency_new,frequency_pc_new,frequency_old,frequency_pc_old,freq_diff
35,healthcare,healthcare - general,clinic,0.0,0.0,1.0,0.374532,-1.0
42,leisure,leisure - general,dance,0.0,0.0,1.0,0.374532,-1.0
65,shop,shop - clothes,tailor,0.0,0.0,1.0,0.374532,-1.0
70,shop,shop - food,deli,0.0,0.0,1.0,0.374532,-1.0
76,shop,shop - food,seafood,0.0,0.0,1.0,0.374532,-1.0
84,shop,shop - for the body,perfumery,0.0,0.0,1.0,0.374532,-1.0
92,shop,shop - general,car,0.0,0.0,1.0,0.374532,-1.0
99,shop,shop - general,curtain,0.0,0.0,1.0,0.374532,-1.0
102,shop,shop - general,electronics,0.0,0.0,1.0,0.374532,-1.0


This looks OK: all of the "lost" tags occurred only once in the original data. Most of them have disappeared because the relevant POI no longer exists - I recognise many of them from memory. A few have been replaced by more precise tags.

Now, the same check for the tags that have newly appeared:

In [10]:
df_tags[df_tags['frequency_old'] == 0]

,feature_key,feature_group,feature_value,frequency_new,frequency_pc_new,frequency_old,frequency_pc_old,freq_diff
10,amenity,amenity - general,dancing_school,1.0,0.255102,0.0,0.0,1.0
12,amenity,amenity - general,events_venue,2.0,0.510204,0.0,0.0,2.0
16,amenity,amenity - general,music_school,1.0,0.255102,0.0,0.0,1.0
19,amenity,amenity - general,prep_school,3.0,0.765306,0.0,0.0,3.0
21,amenity,amenity - general,social_facility,3.0,0.765306,0.0,0.0,3.0
25,amenity,amenity - general,trade_school,1.0,0.255102,0.0,0.0,1.0
27,craft,craft - general,ceramic_painting,1.0,0.255102,0.0,0.0,1.0
30,craft,craft - general,electrician,1.0,0.255102,0.0,0.0,1.0
32,craft,craft - general,glaziery,1.0,0.255102,0.0,0.0,1.0
33,craft,craft - general,photographer,4.0,1.020408,0.0,0.0,4.0


Again, everything looks fine here. I remember creating most of those tags.

Let's summarise the change in tag frequency by group:

In [11]:
df_groups = df_tags[['feature_group',
                     'frequency_new', 'frequency_old', 'freq_diff']]
df_groups = df_groups.groupby('feature_group').sum()
df_groups['perc_diff'] = (df_groups['freq_diff'] /
                          df_groups['frequency_old']) * 100
df_groups

,frequency_new,frequency_old,freq_diff,perc_diff
feature_group,,,,
amenity - food+drinks,56.0,49.0,7.0,14.285714
amenity - general,48.0,33.0,15.0,45.454545
craft - general,14.0,3.0,11.0,366.666667
healthcare - general,26.0,17.0,9.0,52.941176
leisure - general,16.0,13.0,3.0,23.076923
office - general,42.0,17.0,25.0,147.058824
shop - clothes,15.0,17.0,-2.0,-11.764706
shop - food,52.0,45.0,7.0,15.555556
shop - for the body,51.0,24.0,27.0,112.500000


Some groups were much better mapped than others a couple of months ago. Amenities serving food and shops selling mainly food both seem to have been fairly well covered, whereas a lot of office and craft POIs had flown under the radar. 

I suspect the large increase in "for the body" is due to hairdressers and beauty salons - let's check that:

In [12]:
df_tags[df_tags['feature_value'].isin(['hairdresser', 'beauty'])]

,feature_key,feature_group,feature_value,frequency_new,frequency_pc_new,frequency_old,frequency_pc_old,freq_diff
78,shop,shop - for the body,beauty,9.0,2.295918,6.0,2.247191,3.0
81,shop,shop - for the body,hairdresser,29.0,7.397959,9.0,3.370787,20.0


As I suspected, a lot of hair/beauty salons have been added, increasing their combined count from 15 to 38.

In [13]:
df_tags[df_tags['feature_group'] == 'office - general']

,feature_key,feature_group,feature_value,frequency_new,frequency_pc_new,frequency_old,frequency_pc_old,freq_diff
48,office,office - general,accountant,1.0,0.255102,0.0,0.000000,1.0
49,office,office - general,architect,1.0,0.255102,0.0,0.000000,1.0
50,office,office - general,association,4.0,1.020408,1.0,0.374532,3.0
51,office,office - general,company,6.0,1.530612,2.0,0.749064,4.0
52,office,office - general,coworking,1.0,0.255102,0.0,0.000000,1.0
53,office,office - general,diplomatic,1.0,0.255102,1.0,0.374532,0.0
54,office,office - general,educational_institution,1.0,0.255102,1.0,0.374532,0.0
55,office,office - general,employment_agency,1.0,0.255102,0.0,0.000000,1.0
56,office,office - general,energy_supplier,1.0,0.255102,1.0,0.374532,0.0
57,office,office - general,estate_agent,9.0,2.295918,7.0,2.621723,2.0


## Comparing individual POIs

In Part 1 I made sure to give the DataFrame a meaningful index, representing a unique object in OSM data. Now I can use this to join the two DataFrames. Again I will use an outer join, so that the new DataFrame will have a row for every POI that exists in either dataset.

In [14]:
df = pd.merge(df_new, df_old, how='outer', on='full_id',
              suffixes=('_new', '_old'), indicator=True)
print('\n', df.columns)
print('\n', len(df))


 Index(['feature_key_new', 'feature_group_new', 'feature_value_new', 'name_new',
       'addr:housenumber_new', 'addr:postcode_new', 'addr:street_or_place_new',
       'brand_new', 'brand:wikidata_new', 'check_date_new', 'lat_new',
       'lon_new', 'osm_link_new', 'feature_key_old', 'feature_group_old',
       'feature_value_old', 'name_old', 'addr:housenumber_old',
       'addr:postcode_old', 'addr:street_or_place_old', 'brand_old',
       'brand:wikidata_old', 'check_date_old', 'lat_old', 'lon_old',
       'osm_link_old', '_merge'],
      dtype='object')

 408


The combined DataFrame has double the number of columns: a "_new" and "_old" version of each original column. That feels like it might be overkill. But any OSM tag might have changed, even latitude and longitude (in fact I did improve the positioning of quite a few points while surveying). So I'm going to keep all the columns for the moment.

Now I have 408 rows - let's see where they came from:

In [15]:
df.value_counts('_merge')

_merge
both          251
left_only     141
right_only     16
Name: count, dtype: int64

So I lost 16 POIs and gained 141. But I'm also interested in POIs that changed significantly over the period. I want to look for changes in the type of POI, and also changes of name (likely to reflect a change of ownership or a rebranding). 

Define and apply a function to identify the different types of possible change:

In [16]:
def assign_status(row):
    if row['_merge'] == 'left_only':
        return 'ADDED'
    elif row['_merge'] == 'right_only':
        return 'REMOVED'
    elif (row['feature_value_new'] != row['feature_value_old']
          and (row['name_old'].lower() not in row['name_new'].lower())):
        return 'POI TYPE CHANGE'
        # change in feature type (but assume that if name has not changed or
        # is contained within the new name, then it's probably a tagging
        # refinement rather than a real change)
    elif (row['name_old'] != ''
          and (row['name_old'].lower() not in row['name_new'].lower())):
        return 'NAME CHANGE'
        # name has changed (excluding where not previously set, also try
        # to exclude cases where name has just been expanded)
    else:
        return 'NO CHANGE'


df.insert(0, 'comparison', '')
df['comparison'] = df.apply(assign_status, axis=1)

df.value_counts('comparison')

comparison
NO CHANGE          205
ADDED              141
POI TYPE CHANGE     28
NAME CHANGE         18
REMOVED             16
Name: count, dtype: int64

Take a closer look at the name changes:

In [17]:
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.width', 120)

df_name = df[df['comparison'] == 'NAME CHANGE'][['name_new', 'name_old']]
df_name

,name_new,name_old
full_id,,
node/12077811304,Pizza e Amore,Pizza Amore
node/3473031749,La Esquinita del Chupa y Tira,Esquinta del Chupa y Tira
node/4855451338,Kantis,Silvia Bassino
node/4855493937,La Ventana,Panadería La Merced
node/4856185580,Rey Istanbul Kebab,Victoria Kebab
node/4856204293,Grano Puro,El Tapeo del Compás
node/4856204294,Bodeguilla El Hueco,Bar la Comba
node/4857027144,Pablo Compón,La Quinta Generación
node/4857027146,La Finca,Ocasión Casa


Most of these are meaningful name changes, but some are clearly just spelling corrections that I'd prefer not to classify as a data change. Python's difflib.SequenceMatcher class may help:

In [18]:
from difflib import SequenceMatcher

# Function to calculate similarity


def calculate_similarity(row):
    return SequenceMatcher(None, row['name_new'], row['name_old']).ratio()


# Add similarity column to df
df_name['name_similarity'] = df_name.apply(calculate_similarity, axis=1)
df_name.sort_values('name_similarity', ascending=False)

,name_new,name_old,name_similarity
full_id,,,
node/3473031749,La Esquinita del Chupa y Tira,Esquinta del Chupa y Tira,0.925926
node/12077811304,Pizza e Amore,Pizza Amore,0.916667
node/8329995439,Consulado Honorario de Costa Rica,Consulado General de Costa Rica,0.843750
node/9151074711,Apartamentos Fernando el Católico,Fernando el Catolico,0.716981
node/4856185580,Rey Istanbul Kebab,Victoria Kebab,0.500000
node/4861908443,Carnícas Matahambre,Carnicería Miguel,0.500000
node/4862183021,Waffa Moda,Arreglos Waffa,0.416667
node/4856204294,Bodeguilla El Hueco,Bar la Comba,0.322581
way/493911188,Barbatuke Bed & Bike,Casa Babylon Backpackers Hostel,0.313725


Admittedly this is a very small sample, but it looks like a ratio soemwhere above 0.5 and below 0.7 would help distinguish trivial from meangingful changes. The suggested threshold in the Python documentation is 0.6, so I'll add a check for that into the rewritten function:

In [19]:
def assign_status(row):
    if row['_merge'] == 'left_only':
        return 'ADDED'
    elif row['_merge'] == 'right_only':
        return 'REMOVED'
    elif (row['feature_value_new'] != row['feature_value_old']
          and (row['name_old'].lower() not in row['name_new'].lower())):
        # change in feature type (but assume that if name has not changed or
        # is contained within the new name, then it's probably a tagging
        # refinement rather than a real change)
        return 'POI TYPE CHANGE'
    elif (row['name_old'] != ''
          and (row['name_old'].lower() not in row['name_new'].lower())
          and SequenceMatcher(None, row['name_new'], row['name_old']).ratio() < 0.6
          ):
        # name has changed (excluding where not previously set, also try
        # to exclude cases where name has just been expanded)
        return 'NAME CHANGE'
    else:
        return 'NO CHANGE'


df['comparison'] = df.apply(assign_status, axis=1)

df.value_counts('comparison')

comparison
NO CHANGE          209
ADDED              141
POI TYPE CHANGE     28
REMOVED             16
NAME CHANGE         14
Name: count, dtype: int64

In [20]:
print(df[df['comparison'] == 'NAME CHANGE'][['name_new', 'name_old']])
print()
print(df[df['comparison'] == 'REMOVED'][['feature_value_old', 'name_old']])
print()
print(df[df['comparison'] == 'POI TYPE CHANGE'][[
    'feature_value_new', 'feature_value_old', 'name_new', 'name_old']])

                             name_new                         name_old
full_id                                                               
node/4855451338                Kantis                   Silvia Bassino
node/4855493937            La Ventana              Panadería La Merced
node/4856185580    Rey Istanbul Kebab                   Victoria Kebab
node/4856204293            Grano Puro              El Tapeo del Compás
node/4856204294   Bodeguilla El Hueco                     Bar la Comba
node/4857027144          Pablo Compón             La Quinta Generación
node/4857027146              La Finca                     Ocasión Casa
node/4861908435   La Casita de Maykel                   Mesón Victoria
node/4861908437              Nicolino                       Tramontana
node/4861908443   Carnícas Matahambre                Carnicería Miguel
node/4862183021            Waffa Moda                   Arreglos Waffa
node/5274177525     Genoveses Chicano          HC Peluqería y Estética
node/5

The REMOVED, NAME CHANGE, and POI TYPE change listings all look OK. There are still a handful of POIs where the change is really a mapping improvement rather than a real-world change, but it would be difficult to remove those programmatically.

As it stands the data is not very well organised for plotting, as key information such as latitude and longitude is spread across the "_new" and "_old" columns. Before saving the data to CSV, I'm going to add a combined version of a few key columns. These columns will use the "_old" value for POIs that have been removed, and the "_new" value for everything else. 

In [21]:
plot_cols = ['lat', 'lon', 'name', 'feature_value']
plot_cols_new = ['lat_new', 'lon_new', 'name_new', 'feature_value_new']
plot_cols_old = ['lat_old', 'lon_old', 'name_old', 'feature_value_old']
df[plot_cols] = df[plot_cols_new].mask(
    df['comparison'] == 'REMOVED', df[plot_cols_old], axis=0)
list(df.columns)

['comparison',
 'feature_key_new',
 'feature_group_new',
 'feature_value_new',
 'name_new',
 'addr:housenumber_new',
 'addr:postcode_new',
 'addr:street_or_place_new',
 'brand_new',
 'brand:wikidata_new',
 'check_date_new',
 'lat_new',
 'lon_new',
 'osm_link_new',
 'feature_key_old',
 'feature_group_old',
 'feature_value_old',
 'name_old',
 'addr:housenumber_old',
 'addr:postcode_old',
 'addr:street_or_place_old',
 'brand_old',
 'brand:wikidata_old',
 'check_date_old',
 'lat_old',
 'lon_old',
 'osm_link_old',
 '_merge',
 'lat',
 'lon',
 'name',
 'feature_value']

Now I can go ahead and generate some outputs.

In [22]:
df.to_csv(output_file_csv)

In [23]:
# Create a scatter plot
def create_scatter_plot(df, date_new, date_old):
    fig = px.scatter_map(
        df,
        lat='lat',
        lon='lon',
        hover_name='name',
        hover_data={
            'lat': False,
            'lon': False,
            'comparison': True,
            'feature_value': True,
        },
        color='comparison',
        zoom=15
    )

    # some formatting
    fig.update_traces(
        marker=dict(size=10),
    )

    # some more formatting
    fig.update_layout(
        title=(f'Changes in POIs in La Victoria and nearby barrios '
               f'({date_old} to {date_new})'),
        legend_title='COMPARISON STATUS',
        margin=dict(l=10, r=10, b=10)
    )

    return fig

In [24]:
# Create the plot
fig = create_scatter_plot(df, date_new, date_old)

# save as html and display
fig.write_html(output_file_html)
fig.show()

As I mentioned in Part 1, I've found the behaviour of Plotly figures within Jupyter Notebook difficult to control. If nothing appears here, open the output HTML file.

## Comparing over a longer time period

The changes identified in the above analysis mainly relate to recent mapping activity, which has caught up with real-world changes over a longer period. I'd also like to look at changes over a longer period of 5 years, so I'm going to replace the "old" DataFrame with earlier data.

In [25]:
date_old = '2019-10-29'
df_old = get_POIs(date_old)
len(df_old)

237

There were 237 POIs in October 2019, compared to 267 and in August 2024 and 392 now. So mappers weren't completely inactive here up to August 2024 but new POIs were added relatively slowly. I can repeat the previous analysis, now looking at changes from October 2019 to October 2024:

In [26]:
def get_combined_df(df_new1, df_old1):
    # merge the two DataFrames
    df = pd.merge(df_new1, df_old1, how='outer', on='full_id',
                  suffixes=('_new', '_old'), indicator=True)

    # assign status to each row
    df.insert(0, 'comparison', '')
    df['comparison'] = df.apply(assign_status, axis=1)

    # create columns for plotting, using new value where available
    plot_cols = ['lat', 'lon', 'name', 'feature_value']
    plot_cols_new = ['lat_new', 'lon_new', 'name_new', 'feature_value_new']
    plot_cols_old = ['lat_old', 'lon_old', 'name_old', 'feature_value_old']
    df[plot_cols] = df[plot_cols_new].mask(
        df['comparison'] == 'REMOVED', df[plot_cols_old], axis=0)

    return df

In [27]:
df = get_combined_df(df_new, df_old)
df.value_counts('comparison')

comparison
ADDED              197
NO CHANGE          135
REMOVED             42
POI TYPE CHANGE     38
NAME CHANGE         22
Name: count, dtype: int64

In [28]:
output_file = 'compare-data-' + date_old + '-to-' + date_new
output_file_csv = os.path.join(output_folder, output_file + '.csv')
output_file_html = os.path.join(output_folder, output_file + '.html')

df.to_csv(output_file_csv)

I already know that many POIs that existed in 2019 were not mapped until recently, so I can't tell much from the totals. But I might be able to see some patterns in the POIs that were already mapped in 2019. I'm going to create a and plot a second DataFrame containing only those original points.

In [29]:
df2 = df[df['comparison'] != 'ADDED']
print(len(df2))
print()
print(df2.value_counts('comparison'))

237

comparison
NO CHANGE          135
REMOVED             42
POI TYPE CHANGE     38
NAME CHANGE         22
Name: count, dtype: int64


In [30]:
# Create the plot
fig2 = create_scatter_plot(df2, date_new, date_old)

# save as html and display
fig2.write_html(output_file_html)
fig2

Did some kinds of POI change more rapidly than others? Let's summarise the changes:

In [31]:
df2_summary = df2.groupby(
    ['feature_key_old', 'feature_group_old', 'feature_value_old', 'comparison']).size()
df2_summary.head()

feature_key_old  feature_group_old      feature_value_old  comparison     
amenity          amenity - food+drinks  bar                NO CHANGE          2
                                        cafe               NO CHANGE          9
                                                           POI TYPE CHANGE    2
                                                           REMOVED            2
                                        fast_food          NAME CHANGE        2
dtype: int64

It would be easier to interpret this summary DataFrame if the counts for each type of change were in columns. Unstacking the columns:

In [32]:
df2_summary = df2_summary.unstack(fill_value=0)
df2_summary['ORIGINAL'] = df2_summary.sum(axis=1)
df2_summary.head()

comparison                                               NAME CHANGE  NO CHANGE  POI TYPE CHANGE  REMOVED  ORIGINAL
feature_key_old feature_group_old     feature_value_old                                                            
amenity         amenity - food+drinks bar                          0          2                0        0         2
                                      cafe                         0          9                2        2        13
                                      fast_food                    2          2                1        2         7
                                      ice_cream                    1          1                0        0         2
                                      restaurant                   4          8                4        1        17

Now we can look at the changes for each feature group:

In [33]:
df2_group_summary = df2_summary.groupby(['feature_group_old']).sum()
df2_group_summary['unchanged_pc'] = df2_group_summary['NO CHANGE'] / \
    df2_group_summary['ORIGINAL'] * 100
df2_group_summary

comparison,NAME CHANGE,NO CHANGE,POI TYPE CHANGE,REMOVED,ORIGINAL,unchanged_pc
feature_group_old,,,,,,
amenity - food+drinks,7,22,7,5,41,53.658537
amenity - general,0,21,2,8,31,67.741935
craft - general,0,1,0,0,1,100.000000
healthcare - general,1,13,1,0,15,86.666667
leisure - general,2,3,0,2,7,42.857143
office - general,3,4,1,1,9,44.444444
shop - clothes,0,7,5,6,18,38.888889
shop - food,4,28,9,10,51,54.901961
shop - for the body,3,15,3,3,24,62.500000


Generally there is been a significant rate of change across all the main groups. Clothes shops stand out as especially prone to change, and healthcare as more stable. Let's look at the individual POIs in both groups:

In [34]:
df2[df2['feature_group_old'] == 'shop - clothes'][['comparison',
                                                   'feature_value_new', 'name_new', 'feature_value_old', 'name_old']]

,comparison,feature_value_new,name_new,feature_value_old,name_old
full_id,,,,,
node/4855317181,POI TYPE CHANGE,art,El Escaparate,clothes,Modas Valdi
node/4855317186,NO CHANGE,clothes,Mercería Reyes,clothes,Mercería Reyes
node/4855317188,NO CHANGE,clothes,Veneno Moda,clothes,Veneno Moda
node/4855451339,REMOVED,NaN,NaN,clothes,The Park
node/4855451341,REMOVED,NaN,NaN,clothes,
node/4855493934,POI TYPE CHANGE,fitness_centre,Upgrade,clothes,Feeling Modas
node/4856539946,NO CHANGE,clothes,Cristina Modas,clothes,Cristina Modas
node/4856539948,REMOVED,NaN,NaN,clothes,Concha González
node/4856539953,POI TYPE CHANGE,pet_grooming,El Rincón de Luna,clothes,Estela y Lola


In [35]:
df2[df2['feature_group_old'] == 'healthcare - general'][['comparison',
                                                         'feature_value_new', 'name_new', 'feature_value_old', 'name_old']]

,comparison,feature_value_new,name_new,feature_value_old,name_old
full_id,,,,,
node/4728164415,NO CHANGE,pharmacy,Farmacia Ferrándiz,pharmacy,Farmacia Ferrándiz
node/4728164416,NO CHANGE,pharmacy,Farmacia Coronado,pharmacy,Farmacia Coronado
node/4728164420,NO CHANGE,pharmacy,Farmacia Plaza de la Victoria,pharmacy,Farmacia Plaza Victoria
node/4796806091,NO CHANGE,pharmacy,Farmacia Laynez,pharmacy,
node/4855451338,NAME CHANGE,dentist,Kantis,dentist,Silvia Bassino
node/4855451342,NO CHANGE,pharmacy,Farmacia Sanjuan Valero,pharmacy,
node/4856185576,NO CHANGE,dentist,Clínica Dental de la Victoria,dentist,Clínica Dental de la Victoria
node/4856220372,POI TYPE CHANGE,centre,Centro Médico Conductores,clinic,Europreven
node/4856539949,NO CHANGE,dentist,Bucaldent,dentist,Bucaldent


## Next steps

The next steps are more about mapping than coding: having brought these POIs up to date, I need to keep them that way. That should mean that future analyses like this one can focus more on real changes (businesses opening, closing, and changing hands) than on side effects of mapping activity.

In the meantime, I plan to tidy up some of this code and move it to a Python script that I can use to monitor changes in local POIs on a recurring basis.